## Project 2: Food Facts
#### Presented by Carlee Price, Yubo Zhang, and Nikki Haas
#### August 2015


### Introduction

### Part 0: The Questions

1.  How likely is it that a basket of commercially available food items is going to meet recommended levels of nutrients while staying within recommended calorie allowance?
2.  How often does a daily allotment of food contain ingredients known to cause harm to human health, such as high fructose corn syrup and hydrogenated oils?
3.  What are the most common categories of food available from the dataset, and how often are those foods considered nutrient dense?

### Part 1: The Data

Add the other ipynbs here 

  ### Part 2: Statistical Methodology
  
We will use the bootstrapping method and the Student-T method to construct a random set of 10,000 days.  Of those days, we will see how often the day meets, exceeds, or falls short of the FDA guidelines for calorie and nutrient intake.   

The bootstrapping method is a random sampling method with replacement on a set that is assumed to be incomplete.<sup>1</sup>  Numpy's Random selection function allows for with replacement sampling of a set<sup>2</sup> so it is perfect for building a bootstrapping model.  The SciPy module includes a student-T distribution method, so we will be able to use these packages to answer our study questions.<sup>3</sup>

### Part 3: Regulatory Body Recommendations

something about FDA and USDA here

### Part 4: Bias

Bias is introduced in our study via numerous channels.  The entire layout of a grocery store is extremely biased, with soda and snack companies paying top dollar to occupy the most coveted positions and aisles.  Customers have to take extra care to avoid the appealing packages and advertising of calorically dense food and often have to go to seek out the healthy and minially-processed foods. <sup>1</sup> 

Raw foods such as bulk rice, beans, and produce often lack Universal Price Codes (UPCs).  Our datasource is based upon UPCs, and thus may exclude some of the most healthful foods in the average supermarket.  The Food Facts datasource may therefore skew towards potentially unhealthy foods and not reflect what people actually purchase at the grocery store.

Food Facts' collection methods also allow for bias.  The database is built upon an opt-in model of user contributed data.  Contributors both have to know about the project and take the time and effort from their day to upload images, UPCs, and packaging details to Food Facts.  This will bias against consumers who are unable to afford the time or luxury of a smart phone to contribute. <sup>2</sup> 

There is personal bias from the preparers of this study.  One of the authors is a plant based vegetarian, and another is a food scientist.  Our unique perspectives may introduce bias to our findings.  All of the authors reside  in the United States and thus our findings may not hold true in all markets.

### Part 5: Limitations & Shopping Habits

The Open Food Facts' database is populated by consumers who opt in. It does not include a comprehensive list of what is available in the typical grocery store, nor does it represent all foods available in all markets.  In addition, the package sizing data is not consistent for all products.  Some products mention their package contains 800 grams, while other products mention they contain six servings.  The data may or may not include a serving size, and if it does, the units of measure are not consistent.

To mitigate these limitations, we have found that the typical US resident consumes roughly 4 to 4.5 pounds of food per day (about 2040 grams per day)<sup>1</sup>, and that the average person purchases 20 items at a grocery store per week<sup>2</sup>, and eats out about six times a week<sup>3</sup>.  Quite a bit of the set contains data on nutrition information per 100 grams, so we have been able to determine that each person must purchase 20 5 serving items from a grocery store each week, and that the servings are about 100 grams each.  Thus, a perosn must eat 20 100 gram servings per day.  We will therefore subset the data to exclude products that have no nutritive information listed and randomly select 20 items to represent 1 day's worth of food.  In addition, the calorie field did not exist in the dataset so we had to derive it as a calculated field.  

### References

#### Part 1:  The  data
1. [Open Food Facts](http://world.openfoodfacts.org/)


#### Part 2: Statistical Metholodgy
1. [Introduction to Bootstrapping](https://en.wikipedia.org/wiki/Bootstrapping_(statistics))
2. [Implementing Bootstrapping using Python](http://docs.scipy.org/doc/numpy/reference/generated/numpy.random.choice.html)
3. [Student-T Python Module](http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html)


#### Part 3: Regulatory Body Recommendations
1. [FDA Dietary Guidelines, 2015](https://health.gov/dietaryguidelines/2015/guidelines/)
2. [USDA National Nutrient Database](https://ndb.nal.usda.gov/)

#### Part 4: Bias
1. [The Omnivore's Dilemma by Michael Pollan](http://michaelpollan.com/books/the-omnivores-dilemma/)
2. [Open Food Fact's Terms of Contribution](http://world.openfoodfacts.org/terms-of-use#contribution)

#### Part 5: Limitations
1. [USDA Fact Book](http://www.usda.gov/factbook/chapter2.pdf)
2. [Grocery Shoppers' Habits](http://www.marketingcharts.com/traditional/the-average-grocery-shopper-buys-less-than-1-of-available-items-over-the-course-of-the-year-39360/)
3. [United States Healthful Food Council](http://ushfc.org/about/)